### Import Library

In [27]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_redis import RedisVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [28]:
# Load the Google Generative AI model
loader = TextLoader("data/sample_data.txt")
documents = loader.load()

# split data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

texts = text_splitter.split_documents(documents)

# embed the data
embeddings = GoogleGenerativeAIEmbeddings(
  model="models/text-embedding-004",
  google_api_key=os.getenv("GOOGLE_API_KEY"),
)

# redis vector store
vector_store = RedisVectorStore(
  redis_url= os.getenv("REDIS_URL"),
  index_name="langchain_index",
  embeddings=embeddings,
  index_type="hnsw",
)

vector_store.add_documents(texts)
print("Succesffully added documents to Redis Vector Store")

12:03:11 redisvl.index.index INFO   Index already exists, not overwriting.
Succesffully added documents to Redis Vector Store


In [29]:
vector_store.similarity_search("Isna", k=3)

[Document(metadata={}, page_content='Curriculum Vitae\n\nNama Lengkap: Isnania Ramalia  \nStatus: Mahasiswa aktif  \nProgram Studi: Teknik Informatika  \nUniversitas: Universitas Dian Nuswantoro (Udinus), Semarang  \nSemester: 5  \nDomisili: Semarang  \nEmail: isnanramalia@example.com  \nBlog: https://medium.com/@isnanramalia  \nTujuan Karir: Menjadi Frontend Developer atau UI UX Designer\n\nPengalaman Organisasi\n\nDianus Wantoro Computer Club NCC  \n- Anggota aktif  \n- Kegiatan: pengembangan software, kompetisi pemrograman, pelatihan internal\n\nAsisten Mata Kuliah Bengkel Koding  \n- Membantu mahasiswa memahami materi praktikum pemrograman\n\nProyek dan Pengalaman\n\nLeolit Games Website  \n- Stack: Next.js 15, Tailwind CSS, TypeScript, Bun, ShadCN, Framer Motion  \n- Peran: Frontend Developer dan UI Designer  \n- Fitur: Carousel, tampilan markdown dinamis, table of contents, fitur survei admin dan koordinator'),
 Document(metadata={}, page_content='Curriculum Vitae\n\nNama Lengkap

In [30]:
llm = ChatGoogleGenerativeAI(
  model="gemini-1.5-flash",
  google_api_key=os.getenv("GOOGLE_API_KEY")
  )

In [31]:
llm.invoke("siapa isna?")

AIMessage(content='Saya tidak memiliki informasi tentang siapa Isna.  Isna mungkin nama seseorang, tempat, atau hal lain.  Untuk mendapatkan informasi lebih lanjut, saya memerlukan konteks tambahan.  Bisakah Anda memberikan informasi lebih detail tentang siapa atau apa Isna yang Anda maksud?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--7731761b-dbd2-4a43-8a48-2629b41b66f7-0', usage_metadata={'input_tokens': 5, 'output_tokens': 54, 'total_tokens': 59})

### implementasion

[example web](widyaseni.web.id)

In [33]:
template ="""
anda adalah bot yang bertugas menjawab pertanyaan terkait data diri Isna. 
Ini adalah context yang bisa kamu gunakan untuk menjawab pertanyaan: {context}

Pertanyaan: {question}

"""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    # ("human", "{question}"),
    # ("assistant", "{context}")
    ("user", "{question}"),
])

chain =(
  {
    "context": lambda x: vector_store.similarity_search(x["question"], k=3),
    "question": lambda x: x["question"],
  }
  | prompt
  | llm
)

chain.invoke({"question": "siapa isna?"})

AIMessage(content='Isna, atau Isnania Ramalia, adalah seorang mahasiswi aktif semester 5 di Program Studi Teknik Informatika, Universitas Dian Nuswantoro (Udinus), Semarang.  Ia berdomisili di Semarang dan memiliki tujuan karir untuk menjadi Frontend Developer atau UI UX Designer.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--caf50035-05ce-4465-a50a-37d12ca4f69b-0', usage_metadata={'input_tokens': 820, 'output_tokens': 60, 'total_tokens': 880})